# HKU QIDS 2023 Quantitative Investment Competition: Model

## Init Config

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from qids_package.qids import *
import warnings
from submit import submit
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")

In [12]:
seed = 257248
stock_num = 54
day_num_total = 1000
day_num = 1000 - 2
test_day_num = 700
timeslot_num = 50

In [13]:
def std(train, valid, test=None):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    valid = scaler.transform(valid)
    if test is not None:
        test = scaler.transform(test)
    return train, valid, test

In [14]:
def calc_corr(df1, df2):
    return np.corrcoef(df1, df2)[0][1]

In [15]:
def evaluate(model, train, valid, test, train_y, valid_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = model.predict(train)
        model_valid_y = model.predict(valid)
        pred = model.predict(test)
        acr_train = model.score(train, train_y)
        acr_valid = model.score(valid, valid_y)
        print(calc_corr(train_y, model_train_y))
        print(calc_corr(valid_y, model_valid_y))
        if return_pred:
            return pred

def evaluate2(model, train, test, train_y, real_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = model.predict(train)
        pred = np.array(model.predict(test)).reshape(-1,1)
        pred_temp = pred[:37692]
        real_y = np.array(real_y).reshape(-1,1)
        # print(real_y.shape)
        # print(pred.shape)
        # print("train_score",calc_corr(train_y, model_train_y))
        # print("prediction_score",calc_corr(pred, real_y))
        train_y = pd.DataFrame(train_y)
        model_train_y = pd.DataFrame(model_train_y)
        combined0 = pd.concat([train_y,model_train_y],axis = 1)
        print(combined0.corr())
        pred_temp = pd.DataFrame(pred_temp)
        real_y = pd.DataFrame(real_y)
        combined = pd.concat([pred_temp,real_y],axis = 1)
        print(combined.corr())
        if return_pred:
            return pred
def evaluate3(model, train, test, train_y, real_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
    model.fit(train, train_y)
    if version == 2:
        model_train_y = model.predict(train)
        pred = np.array(model.predict(test)).reshape(-1,1)
        pred_temp = pred[:37692]
        real_y = np.array(real_y).reshape(-1,1)
        # print(real_y.shape)
        # print(pred.shape)
        # print("train_score",calc_corr(train_y, model_train_y))
        # print("prediction_score",calc_corr(pred, real_y))
        train_y = pd.DataFrame(train_y)
        model_train_y = pd.DataFrame(model_train_y)
        combined0 = pd.concat([train_y,model_train_y],axis = 1)
        # print(combined0.corr())
        pred_temp = pd.DataFrame(pred_temp)
        real_y = pd.DataFrame(real_y)
        combined = pd.concat([pred_temp,real_y],axis = 1)
        # print(combined.corr())
        if return_pred:
            return pred, combined.corr().iloc[1,0]


## Load Data

In [24]:
write_path = "../data/"

train_path = write_path + "train.csv"
valid_path = write_path + "valid.csv"
test_path = write_path + "test.csv"
real_return_path = write_path + 'real_return.csv'

train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)
test = pd.read_csv(test_path)
real_return = pd.read_csv(real_return_path)

In [6]:
train.corr()

,stock_id,day,turnoverRatio,transactionAmount,pe_ttm,pe,pb,ps,pcf,open_mean,...,low_mean,volume_mean,money_mean,high_max,volume_max,money_max,low_min,price_diff,price_diff_max,return
stock_id,1.000000e+00,-2.352437e-15,-0.058139,-0.050819,-0.019765,0.061225,-0.149061,-0.142580,-0.035876,0.036396,...,0.036384,-0.106283,-0.009172,0.036443,-0.085505,-0.011931,0.036343,-0.010037,-0.007684,-0.000584
day,-2.352437e-15,1.000000e+00,-0.332743,-0.138436,0.012535,-0.063427,-0.113540,-0.110320,-0.029654,0.074004,...,0.074092,-0.210703,-0.136638,0.073257,-0.168088,-0.119024,0.074878,-0.269589,-0.256722,-0.039767
turnoverRatio,-5.813918e-02,-3.327434e-01,1.000000,0.620331,-0.013631,0.047941,0.119004,0.193394,0.005828,-0.101944,...,-0.102073,0.598869,0.445796,-0.100592,0.537153,0.453859,-0.103462,0.461232,0.442833,-0.009228
transactionAmount,-5.081882e-02,-1.384365e-01,0.620331,1.000000,-0.007899,0.067887,0.175861,0.179436,-0.001227,0.079523,...,0.079379,0.821594,0.771382,0.081019,0.678574,0.695656,0.077892,0.342178,0.318609,-0.037591
pe_ttm,-1.976487e-02,1.253483e-02,-0.013631,-0.007899,1.000000,0.031536,-0.040169,-0.013604,0.003151,-0.013801,...,-0.013794,-0.000013,-0.008811,-0.013853,0.000824,-0.006655,-0.013745,-0.017125,-0.018736,-0.002345
pe,6.122505e-02,-6.342674e-02,0.047941,0.067887,0.031536,1.000000,0.011460,0.123659,0.003921,-0.000937,...,-0.000975,0.024455,0.082708,-0.000602,0.016186,0.063952,-0.001270,0.022307,0.021626,-0.004527
pb,-1.490610e-01,-1.135399e-01,0.119004,0.175861,-0.040169,0.011460,1.000000,0.697802,0.014502,0.246572,...,0.246482,-0.015220,0.274631,0.247321,-0.026313,0.216644,0.245706,0.089705,0.071745,-0.019484
ps,-1.425797e-01,-1.103197e-01,0.193394,0.179436,-0.013604,0.123659,0.697802,1.000000,-0.010966,0.300355,...,0.300288,-0.038059,0.305969,0.300921,-0.040860,0.244450,0.299712,0.065515,0.050696,-0.015925
pcf,-3.587599e-02,-2.965431e-02,0.005828,-0.001227,0.003151,0.003921,0.014502,-0.010966,1.000000,0.001034,...,0.001034,0.007004,0.003289,0.001056,0.005662,0.003112,0.001023,0.005412,0.006425,-0.000343
open_mean,3.639574e-02,7.400429e-02,-0.101944,0.079523,-0.013801,-0.000937,0.246572,0.300355,0.001034,1.000000,...,1.000000,-0.106221,0.560167,0.999965,-0.090130,0.423446,0.999954,-0.067668,-0.082402,-0.000957


In [25]:
# train = pd.concat([train,valid],axis=0)

train_y = train["return"]
# train = train.drop(columns=["return", "date_time",'stock_id','day'])
train = train.iloc[:, [4,14,15,17,18]]

valid_y = valid["return"]
valid = valid.drop(columns=["return", "date_time"])

# test = test.drop(columns=["date_time",'stock_id','day'])
test = test.iloc[:, [4,14,15,17,18]]

real_return = real_return.drop(columns=["date_time"])

In [26]:
train

,transactionAmount,volume_mean,money_mean,volume_max,money_max
0,17229.0,454075.84,1.114925e+07,1234546.0,3.056567e+07
1,18378.0,401517.02,9.601600e+06,853209.0,2.052481e+07
2,13686.0,307969.24,7.275962e+06,708274.0,1.677659e+07
3,22587.0,563240.50,1.344701e+07,1503134.0,3.517868e+07
4,65239.0,2249971.94,5.879807e+07,15974670.0,4.255601e+08
...,...,...,...,...,...
53887,5554.0,128145.22,1.195648e+06,387389.0,3.608737e+06
53888,6417.0,120310.78,1.124447e+06,283148.0,2.642756e+06
53889,5204.0,164545.76,1.554019e+06,615056.0,5.839099e+06
53890,9143.0,281720.62,2.775468e+06,2456106.0,2.435143e+07


## Normalization

In [21]:
# train, valid, test = std(train.iloc[:, 3:-1], valid.iloc[:, 3:-1], test.iloc[:, 3:])

## Model

### Linear Regression

In [37]:
model = LinearRegression()
pred = evaluate2(model, train, test, train_y, real_return)

          return         0
return  1.000000  0.082343
0       0.082343  1.000000
         0        0
0  1.00000  0.03462
0  0.03462  1.00000


In [38]:
pd.DataFrame(pred)

,0
0,0.000878
1,0.001435
2,0.001100
3,0.000901
4,0.000411
...,...
37795,-0.001919
37796,-0.004109
37797,-0.005192
37798,-0.002927


### Ridge Regression

In [39]:
from sklearn.linear_model import Ridge
for i in [1000]:
    reg = Ridge(alpha=i)
    print("i =",i)
    pred = evaluate2(reg, train, test, train_y, real_return)

i = 1000
          return         0
return  1.000000  0.075072
0       0.075072  1.000000
          0         0
0  1.000000  0.044834
0  0.044834  1.000000


In [40]:
reg.coef_

array([ 2.58394170e-04, -5.16976918e-07, -9.76531339e-08, -7.15090267e-08,
       -1.14145898e-04,  3.30491455e-05, -8.96484369e-09,  1.54571231e-03,
        3.59898006e-04,  5.30759923e-04,  1.86827075e-03,  6.28971106e-09,
        1.90195913e-10, -2.86308393e-03, -3.43596266e-10,  2.06286903e-11,
       -1.43039708e-03,  1.53393785e-04,  1.88582165e-04])

### Lasso Regression

In [9]:
from sklearn import linear_model
for i in [7]:
    print("i =",i)
    reg = linear_model.Lasso(alpha=i)
    pred = evaluate2(reg, train, test, train_y, real_return)

i = 7
         return        0
return  1.00000  0.05818
0       0.05818  1.00000
          0         0
0  1.000000  0.064944
0  0.064944  1.000000


In [42]:
reg.coef_

array([ 0.00000000e+00, -2.02156534e-07, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.99780789e-09,
       -6.33834644e-11,  0.00000000e+00, -1.35028012e-10,  2.18917279e-11,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

### Linear SVR

In [43]:
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
regr = make_pipeline(StandardScaler(),LinearSVR(random_state=0, tol=1e-5))
pred = evaluate2(regr, train, test, train_y, real_return)
# evaluate2(model, train, test, train_y,test_y ,return_pred=True, version=2, return_auc=False, plot_auc=False):

          return         0
return  1.000000  0.031914
0       0.031914  1.000000
          0         0
0  1.000000  0.024179
0  0.024179  1.000000


### SVR

In [12]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.1))
pred = evaluate2(regr, train, test, train_y, real_return)
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import GridSearchCV

# # Define the pipeline
# regr = make_pipeline(StandardScaler(), SVR())

# # Define the hyperparameters to search over
# param_grid = {
#     'svr__C': [0.1, 1, 10],
#     'svr__gamma': [0.1, 1, 10],
#     'svr__epsilon': [0.1, 0.01, 0.001],
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(regr, param_grid, cv=5, n_jobs=-1)
# grid_search.fit(train, train_y)

# # Print the best hyperparameters and test score
# print("Best hyperparameters: ", grid_search.best_params_)
# print("Test score: ", grid_search.score(test, real_return))

          return         0
return  1.000000  0.186708
0       0.186708  1.000000
          0         0
0  1.000000 -0.004562
0 -0.004562  1.000000


### Gradient boosting

In [32]:
from sklearn import datasets, ensemble

for l in [0.99]:
    params = {
        "n_estimators": 200,
        "max_depth": 3,
        "min_samples_split": 8,
        "learning_rate": 0.01,
        "loss": "huber",
        "alpha": l,
    }

    reg = ensemble.GradientBoostingRegressor(**params)
    pred = evaluate2(reg, train, test, train_y, real_return)

          return         0
return  1.000000  0.170887
0       0.170887  1.000000
          0         0
0  1.000000  0.015523
0  0.015523  1.000000


### SGDBoosting

In [13]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
score_lst = []
for i in [6508,36400]:
    reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=10000, tol=1e-6,penalty='l1',random_state=i))
    pred, score = evaluate3(reg, train, test, train_y, real_return)
    print(score)
    score_lst.append(score)
print(max(score_lst), score_lst.index(max(score_lst)))

0.0785636093333935
0.0785918459965048
0.0785918459965048 1


In [20]:
pred

array([[ 0.00083867],
       [ 0.00175657],
       [ 0.00125077],
       ...,
       [-0.00226737],
       [-0.00159325],
       [-0.00088492]])

### Random Forest

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression
regr = RandomForestRegressor(max_depth=7, random_state=6508, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=300)
pred = evaluate2(regr, train, test, train_y, real_return)

# Create the parameter grid based on the results of random search 
# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [7, 8, 9, 10],
#     'max_features': [2, 3],
#     'min_samples_leaf': [3, 4, 5],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [100, 200, 300, 1000]
# }

# # Create a based model
# rf = RandomForestRegressor()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, n_jobs = -1, verbose = 2)
# # Fit the grid search to the data
# grid_search.fit(train, train_y)
# grid_search.best_params_

          return         0
return  1.000000  0.231504
0       0.231504  1.000000
          0         0
0  1.000000  0.032785
0  0.032785  1.000000


## Submission

In [10]:
submit(pred[:,0])